<a href="https://colab.research.google.com/github/PrusTalbot/PrusTalbot/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%20%E2%84%962%20(%D0%A7%D0%90%D0%A1%D0%A2%D0%AC%202%20%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Проанализировать данные о клиентах и спрогнозировать вероятность закрытия их депозитов**



In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from google.colab import files
uploaded=files.upload()

Saving CL-Python-2-Extra-Data.csv to CL-Python-2-Extra-Data.csv


In [ ]:
import io
import pandas as pd
df=pd.read_csv(io.BytesIO(uploaded['CL-Python-2-Extra-Data.csv']))
display(df.head(2))

,Id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1001,999.0,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,1002,44.0,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151,1,-1,0,unknown,no


**ЗАМЕНЯЕМ ПРОПУСКИ В age НА СРЕДНЮЮ А В balance НА 0**

In [ ]:
df.info() #Смотрим в каких столбцах пропуски - меньше строк в столбце чем, в Датафрейме
# Это age и balance

In [ ]:
df['age'].fillna(df['age'].mean(),inplace=True)
df['balance'].fillna(0,inplace=True)

In [ ]:
# print(df.min(),df.max()) # Смотрим минимальные и максимальные значения ЧТОБ НАЙТИ ВЫБРОСЫ


In [ ]:
df[df.age<15] # Отфильтровать возраст по условию (МЛАДШЕ 15 ЛЕТ НЕ ДАДУТ КРЕДИТ)

In [ ]:
df[df.age>105] # Отфильтровать возраст по условию (СТАРШЕ 105ЛЕТ НЕ БЕРЕМ)

In [ ]:
index_of_age_anomaly=df[df.age<15].index    # Находим индекс строк, где возраст менее 15 лет
df.drop(index_of_age_anomaly, inplace=True) # Удаляем эти строки по их индексу
index_of_age_anomaly=df[df.age>105].index
df.drop(index_of_age_anomaly, inplace=True)

df.shape # Смотрим количество строк

(45207, 18)

In [ ]:
df.duplicated()

In [ ]:
df[df.duplicated()] # Показать строки в которых есть дубликаты

In [ ]:
df.drop_duplicates(inplace=True)   # Удалили дуБликаты
df[df.duplicated(subset=['age'])]  # Посмотрели наличие дубликатов в столбце age

**НОРМАЛИЗАЦИЯ**

In [ ]:
from sklearn.preprocessing import MinMaxScaler # Импортируем библиотеку
min_max_scaler=MinMaxScaler()               # Инициализируем экземпляр класса
x=df['balance'].values.reshape(-1,1)        # Приводим к двумерному массиву
min_max_scaler.fit(x)                       # Определяем мин и макс параметры с помощью метода fit в переменную min_max_scaler
x_minmax_scaled=min_max_scaler.transform(x) # Нормализуем данные и создаем новую переменную x_minmax_scaled
df['balance_minmax']=x_minmax_scaled        # Создали столбец и внесли туда значения из x_minmax_scaled


**СТАНДАРТИЗАЦИЯ**

In [ ]:
from sklearn.preprocessing import StandardScaler # Импортируем библиотеку
stand_scaler=StandardScaler()               # Инициализируем экземпляр класса
x=df['balance'].values.reshape(-1,1)          # Приводим к двумерному массиву
stand_scaler.fit(x)
x_standart_scaled=stand_scaler.transform(x)   # Стандартизируем данные и создаем новую переменную x_standart_scaled
df['balance_standart']=x_standart_scaled      # Создали столбец и внесли туда значения из x_standart_scaled

In [ ]:
display(df.head(5))

,Id,age,education,default,balance,housing,loan,contact,day,month,...,retired,self-employed,services,student,technician,unemployed,unknown,divorced,married,single
1,1002,44.0,4,0,29.0,yes,no,unknown,5,may,...,0,0,0,0,1,0,0,0,0,1
2,1003,33.0,4,0,2.0,yes,yes,unknown,5,may,...,0,0,0,0,0,0,0,0,1,0
3,1004,47.0,0,0,1506.0,yes,no,unknown,5,may,...,0,0,0,0,0,0,0,0,1,0
4,1005,33.0,0,0,1.0,no,no,unknown,5,may,...,0,0,0,0,0,0,1,0,0,1
5,1006,35.0,8,0,231.0,yes,no,unknown,5,may,...,0,0,0,0,0,0,0,0,1,0


**СЧИТАЕМ СКОЛЬКО И КАКИЕ ЗНАЧЕНИЯ СОДЕРЖАТ СТОЛБЦЫ**

In [ ]:
df['y'].value_counts() # СЧИТАЕМ СКОЛЬКО КАКИХ ЗНАЧЕНИЙ СОДЕРЖИТ СТОЛБЕЦ 'y'

0    39921
1     5286
Name: y, dtype: int64

In [ ]:
df['job'].value_counts()  # --//--    'job'

blue-collar      9732
management       9457
technician       7596
admin.           5171
services         4154
retired          2263
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           937
unknown           288
Name: job, dtype: int64

In [ ]:
df['marital'].value_counts()   # --//--    'marital'

married     27212
single      12788
divorced     5207
Name: marital, dtype: int64

In [ ]:
df['education'].value_counts()

secondary    23200
tertiary     13299
primary       6851
unknown       1857
Name: education, dtype: int64

In [ ]:
df['default'].value_counts()

no     44392
yes      815
Name: default, dtype: int64

**ЗАМЕНЯЕМ КАТЕГОРИАЛЬНЫЕ ЗНАЧЕНИЯ НА ЧИСЛОВЫЕ**

In [ ]:
y_to_replace = {"no": 0, "yes": 1}       # УСЛОВИЕ
df["y"] = df["y"].replace(y_to_replace)  # ЗАМЕНА

job_cat = pd.get_dummies(df.job)         # РАЗНЕСЕМ 12 ЗНАЧЕНИЙ job ПО СТОЛБЦАМ И ЗАПИШЕМ В job_cat
df = pd.concat([df, job_cat], axis = 1)  # ЗАНЕСЕМ job_cat В НАШ ДАТАФРЕЙМ
df.drop("job", inplace = True, axis = 1) # УДАЛИМ ИСХОДНЫЙ СТОЛБЕЦ

marital_cat = pd.get_dummies(df.marital) # АНАЛОГИЧНО КАК И С job
df = pd.concat([df, marital_cat], axis = 1)
df.drop("marital", inplace = True, axis = 1)
# С ОБРАЗОВАНИЕМ ПРИМЕНИМ УСЛОВНЫЙ КОЭФФИЦИЕНТ, Т.К. УРОВЕНЬ ОБРАЗОВАНИЯ ТРУДНО ОЦЕНИТЬ ЛИНЕЙНО
education_to_replace = {"unknown": 0, "primary": 1, "secondary": 4, "tertiary": 8}
df["education"] = df["education"].replace(education_to_replace)

default_to_replace = {"no": 0, "yes": 1}                    # АНАЛОГИЧНО КАК И С "y"
df["default"] = df["default"].replace(default_to_replace)

In [ ]:
display(df.head(5))

,Id,age,education,default,balance,housing,loan,contact,day,month,...,retired,self-employed,services,student,technician,unemployed,unknown,divorced,married,single
1,1002,44.0,4,0,29.0,yes,no,unknown,5,may,...,0,0,0,0,1,0,0,0,0,1
2,1003,33.0,4,0,2.0,yes,yes,unknown,5,may,...,0,0,0,0,0,0,0,0,1,0
3,1004,47.0,0,0,1506.0,yes,no,unknown,5,may,...,0,0,0,0,0,0,0,0,1,0
4,1005,33.0,0,0,1.0,no,no,unknown,5,may,...,0,0,0,0,0,0,1,0,0,1
5,1006,35.0,8,0,231.0,yes,no,unknown,5,may,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df_corr=df.corr() #СОЗДАЛИ КОРРЕЛЯЦИОННЫЙ ДАТАФРЕЙМ
df_corr['y']

In [ ]:
y_max=df_corr['y'].drop(['y'])
print(abs(y_max).max())

0.39457982070928754


In [ ]:
list_columns=list(df_corr.columns)  # ЗАПИСАЛИ НАЗВАНИЯ СТОЛБЦОВ ИЗ КОРЕЛЛЯЦИОННОЙ ТАБЛИЦЫ В СПИСОК list_columns

In [ ]:
for i in df_corr.columns:  # ПЕРЕБИРАЕМ НАЗВАНИЯ СТОЛБЦОВ В КОРЕЛЛЯЦИОННОЙ ТАБЛИЦЕ
  print (i)